In [43]:
import gym
import numpy as np
import time
from IPython import display

In [44]:
env = gym.make('FrozenLake-v0')

In [45]:
env.P[0][3] # Transition model

[(0.3333333333333333, 1, 0.0, False),
 (0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 0, 0.0, False)]

In [46]:
env.observation_space.n

16

In [47]:
env.action_space.n

4

In [48]:
def play(env, policy, render=False):
    state = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = policy[state]
        next_state, reward, done, info = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            env.render()
            time.sleep(0.2)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

In [49]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play(env, policy_0)

(0.0, 10)

In [50]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play(env, policy_1, True)

  (Down)
SFFF
FHFH
FFFH
HFFG


(0.0, 13)

In [51]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play(env, policy_2, True)

  (Down)
SFFF
FHFH
FFFH
HFFG


(0.0, 9)

In [52]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play(env, policy_3, True)

  (Down)
SFFF
FHFH
FFFH
HFFG


(1.0, 8)

In [53]:
def play_multiple_times(env, policy, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, policy)

        if total_reward > 0:
            success += 1
            list_of_steps.append(steps)

    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Average number of steps: {np.mean(list_of_steps)}')

In [54]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play_multiple_times(env, policy_0, 1000)

Number of successes: 0/1000
Average number of steps: nan


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [55]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play_multiple_times(env, policy_1, 1000)

Number of successes: 58/1000
Average number of steps: 11.775862068965518


In [56]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play_multiple_times(env, policy_2, 1000)

Number of successes: 107/1000
Average number of steps: 16.345794392523363


In [57]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play_multiple_times(env, policy_3, 1000)

Number of successes: 729/1000
Average number of steps: 37.32373113854595


In [58]:
def policy_evaluation(env, policy, max_iters=500, gamma=0.9):
    # Initialize the values of all states to be 0
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # Update the value of each state
        for state in range(env.observation_space.n):
            action = policy[state]

            # Compute the q-value of the action
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * prev_v_values[next_state])

            v_values[state] = q_value # update v-value
        
        # Check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break
    
    return v_values

In [59]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
v_values_0 = policy_evaluation(env, policy_0)
print(v_values_0)

Converged at 0-th iteration.
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [60]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
v_values_1 = policy_evaluation(env, policy_1)
print(v_values_1)

Converged at 48-th iteration.
[0.01904157 0.01519815 0.03161906 0.02371389 0.02538879 0.
 0.06648515 0.         0.05924054 0.13822794 0.18999823 0.
 0.         0.21152109 0.56684236 0.        ]


In [61]:
np.all(v_values_1 >= v_values_0)

True

In [62]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
v_values_2 = policy_evaluation(env, policy_2)
print(v_values_2)

Converged at 53-th iteration.
[0.02889625 0.01951972 0.03616977 0.0271268  0.04790519 0.
 0.07391985 0.         0.08288277 0.19339319 0.21022995 0.
 0.         0.35153135 0.62684674 0.        ]


In [63]:
np.all(v_values_2 >= v_values_1)

True

In [64]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
v_values_3 = policy_evaluation(env, policy_3)
print(v_values_3)

Converged at 80-th iteration.
[0.06888666 0.06141097 0.07440714 0.05580443 0.09185068 0.
 0.11220679 0.         0.14543323 0.24749485 0.29961611 0.
 0.         0.37993438 0.63901935 0.        ]


In [65]:
np.all(v_values_3 >= v_values_2)

True

In [66]:
def value_iteration(env, max_iters=500, gamma=0.9):
    # initialize
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # update the v-value for each state
        for state in range(env.observation_space.n):
            q_values = []
            
            # compute the q-value for each action that we can perform at the state
            for action in range(env.action_space.n):
                q_value = 0
                # loop through each possible outcome
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * prev_v_values[next_state])
                
                q_values.append(q_value)
            
            # select the max q-values
            best_action = np.argmax(q_values)
            v_values[state] = q_values[best_action]
        
        # check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break
    
    return v_values

In [67]:
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)

Converged at 79-th iteration.


In [68]:
optimal_v_values

array([0.06888615, 0.06141054, 0.07440682, 0.05580409, 0.09185022,
       0.        , 0.11220663, 0.        , 0.14543286, 0.2474946 ,
       0.29961593, 0.        , 0.        , 0.3799342 , 0.63901926,
       0.        ])

In [69]:
def policy_extraction(env, v_values, gamma=0.9):
    # initialize
    policy = np.zeros(env.observation_space.n, dtype=np.int)

    # loop through each state in the environment
    for state in range(env.observation_space.n):
        q_values = []
        # loop through each action
        for action in range(env.action_space.n):
            q_value = 0
            # loop each possible outcome
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * v_values[next_state])
            
            q_values.append(q_value)
        
        # select the best action
        best_action = np.argmax(q_values)
        policy[state] = best_action
    
    return policy

In [70]:
optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


In [71]:
optimal_policy
play_multiple_times(env, optimal_policy, 1000)

Number of successes: 732/1000
Average number of steps: 36.44672131147541


In [72]:
play_multiple_times(env, policy_1, 1000)

Number of successes: 61/1000
Average number of steps: 12.59016393442623


## Define Policy Iteration  ♻ 

 ### Step 1️⃣ Initialize random policy

In [73]:
optimal_policy_FrozenLake_v0 = np.random.randint(0,3,env.observation_space.n)

### Step 2️⃣ Policy Evaluation:find values For fixed current policy 𝜋

In [74]:
def policy_evaluation(env, policy, max_iters=500, gamma=0.9):
    # Initialize the values of all states to be 0
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # Update the value of each state
        for state in range(env.observation_space.n):
            action = policy[state]

            # Compute the q-value of the action
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                if done == False:
                  reward = -1;
                q_value += prob * (reward + gamma * prev_v_values[next_state])

            v_values[state] = q_value # update v-value
        
        # Check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break
    
    return v_values

### Step 3️⃣ Policy Improvement :get a better policy using policy extraction

In [75]:
def policy_improvement(env, v_values,policy, gamma=0.9):
    # initialize
    v_values_p = np.copy(v_values)
    policy_ans =np.copy(policy)
    n_loop = 0
    # loop through each state in the environment
    for i in range(1000):
      policy_temp = np.copy(policy_ans)
      for state in range(env.observation_space.n):
          q_values = []
          # loop through each action
          for action in range(env.action_space.n):
              q_value = 0
              # loop each possible outcome
              for prob, next_state, reward, done in env.P[state][action]:
                  q_value += prob * (reward + gamma * v_values_p[next_state])
              
              q_values.append(q_value)
          
          # select the best action and update v_value[state]
          best_action = np.argmax(q_values)
          policy_temp[state] = best_action
          v_values_p[state] = np.max(q_values)

      # return 𝜋' if 𝜋' does not change 
      if np.all(policy_ans == policy_temp):
        break
      # update 𝜋 if change
      policy_ans=np.copy(policy_temp)
    return policy_ans

 # Toy games FrozenLake-v0 🧊

In [76]:
# Policy Iteration frozenlake-v0
env = gym.make('FrozenLake-v0')
tstart_PI_FL =time.time()
policy_FrozenLake_v0 = np.random.randint(0,3,env.observation_space.n)
v_values_FrozenLake_v0 = policy_evaluation(env, policy_FrozenLake_v0)
optimal_policy_FrozenLake_v0 = policy_improvement(env, v_values_FrozenLake_v0,policy_FrozenLake_v0, gamma=0.9)
tend_PI_FL =time.time()

# Value Iteration frozenlake-v0
tstart_VI_FL =time.time()
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)
optimal_policy_FrozenLake_v0_2 = policy_extraction(env, optimal_v_values, gamma=0.9)
tend_VI_FL =time.time()

Converged at 42-th iteration.
Converged at 79-th iteration.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


In [77]:
play_multiple_times(env, optimal_policy_FrozenLake_v0, 1000)
play_multiple_times(env, optimal_policy_FrozenLake_v0_2, 1000)

Number of successes: 534/1000
Average number of steps: 32.22659176029963
Number of successes: 737/1000
Average number of steps: 39.03527815468114


 # Toy games FrozenLake8x8-v0 🧊❄

In [83]:
# Policy Iteration frozenlake8x8-v0
tstart_PI_FL8x8 =time.time()
env = gym.make('FrozenLake8x8-v0')
policy_FrozenLake8x8_v0 = np.random.randint(0,3,env.observation_space.n)
v_values_FrozenLake8x8_v0 = policy_evaluation(env, policy_FrozenLake8x8_v0)
optimal_policy_FrozenLake8x8_v0 = policy_improvement(env, v_values_FrozenLake8x8_v0,policy_FrozenLake8x8_v0, gamma=0.9)
tend_PI_FL8x8 =time.time()

# Value Iteration frozenlake8x8-v0
tstart_VI_FL8x8 =time.time()
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)
optimal_policy_FrozenLake8x8_v0_2 = policy_extraction(env, optimal_v_values, gamma=0.9)
tend_VI_FL8x8 =time.time()

Converged at 62-th iteration.
Converged at 117-th iteration.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


In [84]:
play_multiple_times(env, optimal_policy_FrozenLake8x8_v0, 1000)
play_multiple_times(env, optimal_policy_FrozenLake8x8_v0_2, 1000)

Number of successes: 681/1000
Average number of steps: 69.86637298091043
Number of successes: 732/1000
Average number of steps: 72.27322404371584


 # Toy games Taxi-v3 🚔

In [80]:
# Policy Iteration taxi-v3
env = gym.make('Taxi-v3')
tstart_PI_taxi =time.time()
policy_Taxi_v3 = np.random.randint(0,3,env.observation_space.n)
v_values_Taxi_v3 = policy_evaluation(env, policy_Taxi_v3)
optimal_policy_Taxi_v3 = policy_improvement(env, v_values_Taxi_v3,policy_Taxi_v3, gamma=0.9)
tend_PI_taxi =time.time()

# Value Iteration taxi-v3
tstart_VI_taxi =time.time()
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)
optimal_policy_Taxi_v3_2 = policy_extraction(env, optimal_v_values, gamma=0.9)
tend_VI_taxi =time.time()

Converged at 88-th iteration.
Converged at 116-th iteration.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


In [81]:
play_multiple_times(env, optimal_policy_Taxi_v3, 1000)
play_multiple_times(env, optimal_policy_Taxi_v3_2, 1000)

Number of successes: 1000/1000
Average number of steps: 12.935
Number of successes: 1000/1000
Average number of steps: 13.088


# 🌟So sánh Policy Iteration và Value Iteration?
  - Với Value Iteration trong mỗi vòng lặp để tìm ra v_values ta phải duyệt qua tất cả các action tại mỗi trạng thái của trò chơi và chọn ra action có q_value lớn nhất nên thuật toán này sẽ chậm hơn vì tốn O(S^2 * A) cho mỗi lần lặp và "argmax" ở mỗi trạng thái hiếm khi thay đổi khi số vòng lặp đủ lớn và policy thường hội tụ rất lâu
  - với Policy Iteration chỉ tính v_values dựa trên random policy mà không phải duyệt qua tất cả action nên sẽ nhanh hơn Value Iteration sau khi tính được v_value theo random policy là cải tiến Policy nên độ phức tạp thuật toán nhỏ hơn

▶▶ Đựa vào kết quả play multiple times ở các môi trường có thể thấy optimal policy của thuật toán Value Iteration tối ưu hơn 

In [86]:
# compare time
print("Value Iteration - frozenlake-v0 :",tend_VI_FL-tstart_VI_FL)
print("Policy Iteration - frozenlake-v0:",tend_PI_FL-tstart_PI_FL)
print("Value Iteration - frozenlake8x8-v0 :",tend_VI_FL8x8-tstart_VI_FL8x8)
print("Policy Iteration - frozenlake8x8-v0:",tend_PI_FL8x8-tstart_PI_FL8x8)
print("Value Iteration - taxi-v3 :",tend_VI_taxi-tstart_VI_taxi)
print("Policy Iteration - taxi-v3:",tend_PI_taxi-tstart_PI_taxi)

Value Iteration - frozenlake-v0 : 0.023247480392456055
Policy Iteration - frozenlake-v0: 0.013664007186889648
Value Iteration - frozenlake8x8-v0 : 0.11939692497253418
Policy Iteration - frozenlake8x8-v0: 0.0673825740814209
Value Iteration - taxi-v3 : 1.256840467453003
Policy Iteration - taxi-v3: 0.28287625312805176


**➡ Vậy Trong các toy games ở đây thì thuật toán Policy Iteration cho ra kết quả nhanh hơn**